# Lattice Boltzmann Method for Fluid Flow Simulation

## Introduction

The **Lattice Boltzmann Method (LBM)** is a powerful computational technique for simulating fluid dynamics based on mesoscopic kinetic theory. Unlike traditional Navier-Stokes solvers that directly discretize macroscopic equations, LBM operates at an intermediate scale, tracking the evolution of particle distribution functions on a discrete lattice.

## Theoretical Foundation

### The Boltzmann Equation

The continuous Boltzmann equation describes the evolution of the particle distribution function $f(\mathbf{x}, \mathbf{v}, t)$:

$$\frac{\partial f}{\partial t} + \mathbf{v} \cdot \nabla f = \Omega(f)$$

where:
- $f(\mathbf{x}, \mathbf{v}, t)$ is the probability of finding particles at position $\mathbf{x}$ with velocity $\mathbf{v}$ at time $t$
- $\Omega(f)$ is the collision operator describing particle interactions

### Bhatnagar-Gross-Krook (BGK) Approximation

The collision operator is simplified using the BGK single-relaxation-time approximation:

$$\Omega(f) = -\frac{1}{\tau}(f - f^{eq})$$

where $\tau$ is the relaxation time and $f^{eq}$ is the equilibrium distribution function.

### D2Q9 Lattice Model

In two dimensions, we use the D2Q9 lattice with 9 discrete velocities:

$$\mathbf{e}_i = \begin{cases}
(0, 0) & i = 0 \\
(\cos\theta_i, \sin\theta_i) \cdot c & i = 1, 2, 3, 4 \\
(\cos\theta_i, \sin\theta_i) \cdot \sqrt{2}c & i = 5, 6, 7, 8
\end{cases}$$

where $\theta_i = (i-1)\pi/2$ for cardinal directions and $\theta_i = (i-5)\pi/2 + \pi/4$ for diagonal directions.

### Equilibrium Distribution

The Maxwell-Boltzmann equilibrium distribution, discretized for the lattice:

$$f_i^{eq} = w_i \rho \left[ 1 + \frac{\mathbf{e}_i \cdot \mathbf{u}}{c_s^2} + \frac{(\mathbf{e}_i \cdot \mathbf{u})^2}{2c_s^4} - \frac{\mathbf{u} \cdot \mathbf{u}}{2c_s^2} \right]$$

where:
- $w_i$ are the lattice weights: $w_0 = 4/9$, $w_{1-4} = 1/9$, $w_{5-8} = 1/36$
- $c_s = c/\sqrt{3}$ is the lattice speed of sound
- $\rho$ is the fluid density
- $\mathbf{u}$ is the macroscopic velocity

### Macroscopic Variables

The macroscopic density and momentum are computed as moments of the distribution:

$$\rho = \sum_{i=0}^{8} f_i$$

$$\rho \mathbf{u} = \sum_{i=0}^{8} f_i \mathbf{e}_i$$

### Kinematic Viscosity

The kinematic viscosity $\nu$ relates to the relaxation time:

$$\nu = c_s^2 \left(\tau - \frac{1}{2}\right)$$

### Algorithm Steps

1. **Collision**: $f_i^{\text{out}} = f_i - \frac{1}{\tau}(f_i - f_i^{eq})$
2. **Streaming**: $f_i(\mathbf{x} + \mathbf{e}_i \Delta t, t + \Delta t) = f_i^{\text{out}}(\mathbf{x}, t)$
3. **Boundary conditions**: Apply appropriate boundary treatments
4. **Compute macroscopic variables**: Calculate $\rho$ and $\mathbf{u}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm

# Set up plotting style
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14

## D2Q9 Lattice Configuration

We define the discrete velocity set and corresponding weights for the D2Q9 model.

In [ ]:
# D2Q9 lattice velocities
# Index:     0      1      2      3       4      5       6       7       8
# Direction: rest   E      N      W       S      NE      NW      SW      SE
ex = np.array([0,  1,  0, -1,  0,  1, -1, -1,  1])  # x-components
ey = np.array([0,  0,  1,  0, -1,  1,  1, -1, -1])  # y-components

# Lattice weights
w = np.array([4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36])

# Speed of sound squared (for D2Q9)
cs2 = 1/3

# Opposite direction indices for bounce-back
opposite = np.array([0, 3, 4, 1, 2, 7, 8, 5, 6])

print("D2Q9 Lattice Configuration:")
print(f"Velocities (ex): {ex}")
print(f"Velocities (ey): {ey}")
print(f"Weights: {w}")
print(f"Speed of sound squared: cs² = {cs2:.4f}")

## Core LBM Functions

Implementation of the equilibrium distribution, collision, and streaming operations.

In [ ]:
def compute_equilibrium(rho, ux, uy):
    """
    Compute the equilibrium distribution function f^eq.
    
    Parameters:
    -----------
    rho : ndarray
        Fluid density field
    ux, uy : ndarray
        Velocity components
    
    Returns:
    --------
    feq : ndarray
        Equilibrium distribution (9 x Ny x Nx)
    """
    Ny, Nx = rho.shape
    feq = np.zeros((9, Ny, Nx))
    
    # Precompute common terms
    u_sq = ux**2 + uy**2
    
    for i in range(9):
        # e_i · u
        eu = ex[i] * ux + ey[i] * uy
        # f_i^eq formula
        feq[i] = w[i] * rho * (1 + eu/cs2 + eu**2/(2*cs2**2) - u_sq/(2*cs2))
    
    return feq


def collision(f, feq, omega):
    """
    BGK collision step.
    
    Parameters:
    -----------
    f : ndarray
        Current distribution function
    feq : ndarray
        Equilibrium distribution
    omega : float
        Relaxation parameter (1/tau)
    
    Returns:
    --------
    f_out : ndarray
        Post-collision distribution
    """
    return f - omega * (f - feq)


def streaming(f):
    """
    Streaming step: propagate distributions along lattice velocities.
    
    Parameters:
    -----------
    f : ndarray
        Post-collision distribution
    
    Returns:
    --------
    f_streamed : ndarray
        Distribution after streaming
    """
    f_out = np.zeros_like(f)
    
    for i in range(9):
        f_out[i] = np.roll(np.roll(f[i], ex[i], axis=1), ey[i], axis=0)
    
    return f_out


def compute_macroscopic(f):
    """
    Compute macroscopic density and velocity from distribution.
    
    Parameters:
    -----------
    f : ndarray
        Distribution function
    
    Returns:
    --------
    rho : ndarray
        Density field
    ux, uy : ndarray
        Velocity components
    """
    rho = np.sum(f, axis=0)
    ux = np.sum(f * ex.reshape(-1, 1, 1), axis=0) / rho
    uy = np.sum(f * ey.reshape(-1, 1, 1), axis=0) / rho
    
    return rho, ux, uy

## Simulation Setup: Flow Around a Cylinder

We simulate 2D flow around a cylindrical obstacle - a classic benchmark problem that exhibits the **von Kármán vortex street** at appropriate Reynolds numbers.

The Reynolds number is defined as:

$$Re = \frac{U \cdot D}{\nu}$$

where $U$ is the characteristic velocity, $D$ is the cylinder diameter, and $\nu$ is the kinematic viscosity.

In [ ]:
# Domain parameters
Nx = 400        # Number of lattice nodes in x
Ny = 100        # Number of lattice nodes in y

# Cylinder parameters
cx, cy = Nx // 4, Ny // 2  # Cylinder center
r = Ny // 9                 # Cylinder radius

# Flow parameters
u_inlet = 0.08              # Inlet velocity (lattice units)
Re = 100                    # Target Reynolds number

# Compute viscosity and relaxation time from Reynolds number
nu = u_inlet * (2 * r) / Re  # Kinematic viscosity
tau = nu / cs2 + 0.5         # Relaxation time
omega = 1 / tau              # Relaxation parameter

# Time stepping
n_steps = 15000              # Total simulation steps
plot_interval = 3000         # Interval for saving snapshots

print("Simulation Parameters:")
print(f"Domain size: {Nx} x {Ny}")
print(f"Cylinder: center=({cx}, {cy}), radius={r}")
print(f"Reynolds number: Re = {Re}")
print(f"Inlet velocity: u = {u_inlet}")
print(f"Kinematic viscosity: ν = {nu:.6f}")
print(f"Relaxation time: τ = {tau:.4f}")
print(f"Relaxation parameter: ω = {omega:.4f}")
print(f"Total steps: {n_steps}")

In [ ]:
# Create obstacle mask (cylinder)
Y, X = np.meshgrid(np.arange(Ny), np.arange(Nx), indexing='ij')
obstacle = (X - cx)**2 + (Y - cy)**2 < r**2

# Visualize the geometry
plt.figure(figsize=(12, 4))
plt.imshow(obstacle, cmap='binary', origin='lower', aspect='equal')
plt.colorbar(label='Obstacle (1=solid)')
plt.title('Computational Domain with Cylindrical Obstacle')
plt.xlabel('x (lattice units)')
plt.ylabel('y (lattice units)')
plt.tight_layout()
plt.show()

## Boundary Conditions

We implement:
1. **Inlet (left)**: Zou-He velocity boundary condition
2. **Outlet (right)**: Zero-gradient (extrapolation) condition
3. **Walls (top/bottom)**: Periodic boundary conditions
4. **Obstacle**: Bounce-back for no-slip condition

In [ ]:
def apply_inlet_bc(f, rho, u_in):
    """
    Apply Zou-He velocity boundary condition at inlet (left boundary).
    """
    # Compute density at inlet from known distributions
    rho_in = (f[0, :, 0] + f[2, :, 0] + f[4, :, 0] + 
              2 * (f[3, :, 0] + f[6, :, 0] + f[7, :, 0])) / (1 - u_in)
    
    # Unknown distributions (1, 5, 8) - pointing into domain
    f[1, :, 0] = f[3, :, 0] + (2/3) * rho_in * u_in
    f[5, :, 0] = f[7, :, 0] + (1/6) * rho_in * u_in - 0.5 * (f[2, :, 0] - f[4, :, 0])
    f[8, :, 0] = f[6, :, 0] + (1/6) * rho_in * u_in + 0.5 * (f[2, :, 0] - f[4, :, 0])
    
    return f


def apply_outlet_bc(f):
    """
    Apply zero-gradient boundary condition at outlet (right boundary).
    """
    f[:, :, -1] = f[:, :, -2]
    return f


def apply_bounce_back(f, obstacle):
    """
    Apply bounce-back boundary condition on obstacle surface.
    """
    for i in range(9):
        f[i, obstacle] = f[opposite[i], obstacle]
    return f

## Main Simulation Loop

In [ ]:
# Initialize distribution function to equilibrium
rho = np.ones((Ny, Nx))
ux = np.ones((Ny, Nx)) * u_inlet
uy = np.zeros((Ny, Nx))

# Set velocity to zero inside obstacle
ux[obstacle] = 0
uy[obstacle] = 0

# Initialize with equilibrium distribution
f = compute_equilibrium(rho, ux, uy)

# Storage for snapshots
snapshots = []
snapshot_times = []

print("Starting simulation...")
print(f"{'Step':>8} | {'Max velocity':>12} | {'Mean density':>12}")
print("-" * 40)

for step in range(n_steps):
    # Apply boundary conditions
    f = apply_inlet_bc(f, rho, u_inlet)
    f = apply_outlet_bc(f)
    
    # Compute macroscopic variables
    rho, ux, uy = compute_macroscopic(f)
    
    # Set velocity to zero inside obstacle
    ux[obstacle] = 0
    uy[obstacle] = 0
    
    # Compute equilibrium
    feq = compute_equilibrium(rho, ux, uy)
    
    # Collision step
    f = collision(f, feq, omega)
    
    # Apply bounce-back on obstacle
    f = apply_bounce_back(f, obstacle)
    
    # Streaming step
    f = streaming(f)
    
    # Store snapshots
    if step % plot_interval == 0 or step == n_steps - 1:
        u_mag = np.sqrt(ux**2 + uy**2)
        snapshots.append({
            'ux': ux.copy(),
            'uy': uy.copy(),
            'u_mag': u_mag.copy(),
            'rho': rho.copy()
        })
        snapshot_times.append(step)
        print(f"{step:8d} | {u_mag.max():12.6f} | {rho.mean():12.6f}")

print("\nSimulation complete!")

## Visualization of Results

We visualize the velocity magnitude field and the vorticity, which reveals the vortex shedding behind the cylinder.

In [ ]:
def compute_vorticity(ux, uy):
    """
    Compute vorticity: ω = ∂uy/∂x - ∂ux/∂y
    """
    duy_dx = np.gradient(uy, axis=1)
    dux_dy = np.gradient(ux, axis=0)
    return duy_dx - dux_dy


# Create comprehensive visualization
fig, axes = plt.subplots(3, 2, figsize=(14, 12))

# Final state data
final = snapshots[-1]
vorticity = compute_vorticity(final['ux'], final['uy'])

# 1. Velocity magnitude at final time
ax = axes[0, 0]
im = ax.imshow(final['u_mag'], origin='lower', cmap='viridis', aspect='equal')
ax.contour(obstacle, levels=[0.5], colors='white', linewidths=2)
plt.colorbar(im, ax=ax, label='|u|')
ax.set_title(f'Velocity Magnitude (t = {snapshot_times[-1]})')
ax.set_xlabel('x')
ax.set_ylabel('y')

# 2. Vorticity field
ax = axes[0, 1]
vmax = np.abs(vorticity).max() * 0.5
im = ax.imshow(vorticity, origin='lower', cmap='RdBu_r', 
               vmin=-vmax, vmax=vmax, aspect='equal')
ax.contour(obstacle, levels=[0.5], colors='black', linewidths=2)
plt.colorbar(im, ax=ax, label='ω')
ax.set_title('Vorticity Field (von Kármán vortex street)')
ax.set_xlabel('x')
ax.set_ylabel('y')

# 3. Streamlines
ax = axes[1, 0]
Y_grid, X_grid = np.meshgrid(np.arange(Ny), np.arange(Nx), indexing='ij')
speed = final['u_mag'].copy()
speed[obstacle] = np.nan
ax.imshow(speed, origin='lower', cmap='plasma', aspect='equal', alpha=0.6)
# Create streamplot with masked obstacle
ux_plot = final['ux'].copy()
uy_plot = final['uy'].copy()
ux_plot[obstacle] = np.nan
uy_plot[obstacle] = np.nan
ax.streamplot(np.arange(Nx), np.arange(Ny), ux_plot, uy_plot, 
              color='white', linewidth=0.5, density=2, arrowsize=0.5)
ax.contour(obstacle, levels=[0.5], colors='red', linewidths=2)
ax.set_title('Streamlines')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_xlim(0, Nx)
ax.set_ylim(0, Ny)

# 4. Density field
ax = axes[1, 1]
rho_plot = final['rho'].copy()
rho_plot[obstacle] = np.nan
im = ax.imshow(rho_plot, origin='lower', cmap='coolwarm', aspect='equal')
ax.contour(obstacle, levels=[0.5], colors='black', linewidths=2)
plt.colorbar(im, ax=ax, label='ρ')
ax.set_title('Density Field')
ax.set_xlabel('x')
ax.set_ylabel('y')

# 5. Velocity profiles at different x-locations
ax = axes[2, 0]
x_locations = [cx + 2*r, cx + 5*r, cx + 10*r]
colors = ['blue', 'green', 'red']
for i, x_loc in enumerate(x_locations):
    if x_loc < Nx:
        ax.plot(final['ux'][:, x_loc], np.arange(Ny), 
                color=colors[i], label=f'x = {x_loc}')
ax.axhline(cy, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('$u_x$')
ax.set_ylabel('y')
ax.set_title('Horizontal Velocity Profiles')
ax.legend()
ax.grid(True, alpha=0.3)

# 6. Time evolution of velocity at a probe point
ax = axes[2, 1]
probe_x, probe_y = cx + 3*r, cy + r//2  # Probe in wake region
uy_history = [s['uy'][probe_y, probe_x] for s in snapshots]
ax.plot(snapshot_times, uy_history, 'b-o', markersize=4)
ax.set_xlabel('Time step')
ax.set_ylabel('$u_y$')
ax.set_title(f'Vertical Velocity at Probe ({probe_x}, {probe_y})')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Physical Interpretation

### Von Kármán Vortex Street

At Reynolds numbers around $Re \sim 100$, the flow past a cylinder becomes unstable and develops a **periodic vortex shedding pattern** known as the von Kármán vortex street. This phenomenon is characterized by:

1. **Alternating vortices**: Vortices of opposite rotation shed alternately from the top and bottom of the cylinder
2. **Strouhal number**: The shedding frequency follows $St = \frac{f D}{U} \approx 0.2$ for circular cylinders
3. **Wake oscillation**: The transverse velocity component oscillates sinusoidally in the wake

### Lattice Boltzmann Advantages

The LBM approach offers several advantages for this simulation:

- **Simple boundary handling**: Bounce-back naturally implements no-slip conditions
- **Inherent parallelism**: Local collision and streaming operations are highly parallelizable
- **Complex geometries**: Irregular boundaries are easily handled on regular lattices
- **Compressibility effects**: The method naturally captures weakly compressible flows

In [ ]:
# Compute approximate Strouhal number from velocity oscillations
if len(snapshots) > 2:
    # Use probe point data
    uy_probe = np.array([s['uy'][probe_y, probe_x] for s in snapshots])
    times = np.array(snapshot_times)
    
    # Simple zero-crossing analysis for period estimation
    uy_mean = uy_probe.mean()
    crossings = np.where(np.diff(np.sign(uy_probe - uy_mean)))[0]
    
    if len(crossings) >= 2:
        # Estimate period from crossings
        avg_half_period = np.mean(np.diff(times[crossings]))
        period = 2 * avg_half_period
        frequency = 1 / period
        strouhal = frequency * (2 * r) / u_inlet
        
        print(f"\nApproximate Flow Statistics:")
        print(f"Estimated shedding period: {period:.1f} time steps")
        print(f"Estimated frequency: {frequency:.6f} (per time step)")
        print(f"Strouhal number: St ≈ {strouhal:.3f}")
        print(f"(Expected value for Re=100: St ≈ 0.16-0.20)")
    else:
        print("\nNote: Not enough oscillation cycles captured for Strouhal number estimation.")
        print("Consider running longer simulation for vortex shedding analysis.")

## Conclusions

This notebook demonstrated the implementation of the **Lattice Boltzmann Method** for simulating 2D fluid flow around a cylindrical obstacle. Key takeaways:

1. **Mesoscopic approach**: LBM bridges microscopic kinetic theory and macroscopic fluid dynamics
2. **Simple algorithm**: The collision-streaming paradigm is conceptually elegant and easy to implement
3. **Physical fidelity**: The method correctly captures complex phenomena like vortex shedding
4. **Extensibility**: The framework easily extends to 3D, multi-phase flows, and thermal effects

### References

1. Chen, S., & Doolen, G. D. (1998). Lattice Boltzmann method for fluid flows. *Annual Review of Fluid Mechanics*, 30(1), 329-364.
2. Krüger, T., et al. (2017). *The Lattice Boltzmann Method: Principles and Practice*. Springer.
3. Succi, S. (2001). *The Lattice Boltzmann Equation for Fluid Dynamics and Beyond*. Oxford University Press.